### author by yangshichen
### 注意：脚本仅供参考，使用前请仔细阅读

### 调用GPU

In [3]:
import os
import glob
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import seaborn as sb
from matplotlib.pyplot import rc_context
import matplotlib.pyplot as plt
from scipy.io import mmread
from scipy.sparse import csr_matrix
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import warnings
warnings.filterwarnings("ignore")

In [4]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=100, facecolor='white')

scanpy==1.9.3 anndata==0.8.0 numpy==1.21.6 scipy==1.7.3 pandas==1.3.5 scikit-learn==1.0.2 statsmodels==0.13.2 python-igraph==0.10.8 pynndescent==0.5.13


### 通路评分

In [5]:
adata = sc.read('/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/HIV-pbmc/Myeloid_celltype.h5ad')
adata

AnnData object with n_obs × n_vars = 353468 × 21679
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'doublet_score', 'doublet', 'batch', 'sample', 'stage', 'experiments', 'age', 'total_counts_rp', 'pct_counts_rp', 'total_counts_hb', 'pct_counts_hb', 'total_counts_ncRNA', 'pct_counts_ncRNA', 'total_counts_LOC', 'pct_counts_LOC', 'total_counts_erccs', 'pct_counts_erccs', 'leiden_r2.0_n2', 'celltype_L1', 'phase_ordered', 'celltype_r2.0', 'celltype_L3'
    var: 'hb', 'rp', 'ncRNA', 'LOC', 'erccs', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches', 'mean', 'std', 'dispersions', 'dispersions_norm', 'highly_variable_intersection'
    uns: 'age_colors', 'celltype_r2.0_colors', 'cosg', 'experiments_colors', 'hvg', 'leiden', 'leiden_r1.5_n2_colors', 'leiden_r1.8_n2_colors', 'leiden_r2.0_n2_colors', 'leiden_r2.2_n2_co

In [6]:
gene_sets = {} 
folder_path = '/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/PathwayScore/Geneset/Myeloid/'
txt_files = glob.glob(os.path.join(folder_path, '*.txt'))
for file_path in txt_files:
    with open(file_path, 'r') as file:
        gene_set_name = file.readline().strip()
        gene_list = []
        for line in file:
            gene = line.strip()
            gene_list.append(gene)
        gene_sets[gene_set_name] = gene_list

In [7]:
for gene_set_name, gene_list in gene_sets.items():
    sc.tl.score_genes(adata, gene_list, score_name=gene_set_name)

computing score 'GOBP_CELL_CHEMOTAXIS'
    finished: added
    'GOBP_CELL_CHEMOTAXIS', score of gene set (adata.obs).
    1184 total control genes are used. (0:00:07)
computing score 'GOBP_INFLAMMATORY_RESPONSE'
    finished: added
    'GOBP_INFLAMMATORY_RESPONSE', score of gene set (adata.obs).
    1149 total control genes are used. (0:00:07)
computing score 'HALLMARK_INFLAMMATORY_RESPONSE'
    finished: added
    'HALLMARK_INFLAMMATORY_RESPONSE', score of gene set (adata.obs).
    1189 total control genes are used. (0:00:07)
computing score 'HALLMARK_INTERFERON_ALPHA_RESPONSE'
    finished: added
    'HALLMARK_INTERFERON_ALPHA_RESPONSE', score of gene set (adata.obs).
    749 total control genes are used. (0:00:07)
computing score 'HALLMARK_INTERFERON_GAMMA_RESPONSE'
    finished: added
    'HALLMARK_INTERFERON_GAMMA_RESPONSE', score of gene set (adata.obs).
    989 total control genes are used. (0:00:07)
computing score 'HALLMARK_TNFA_SIGNALING_VIA_NFKB'
    finished: added
    'HAL

In [8]:
gene_set_names = gene_sets.keys()
gene_set_names = [str(name) for name in gene_set_names]
gene_set_names

['GOBP_CELL_CHEMOTAXIS',
 'GOBP_INFLAMMATORY_RESPONSE',
 'HALLMARK_INFLAMMATORY_RESPONSE',
 'HALLMARK_INTERFERON_ALPHA_RESPONSE',
 'HALLMARK_INTERFERON_GAMMA_RESPONSE',
 'HALLMARK_TNFA_SIGNALING_VIA_NFKB',
 'INTERLEUKIN_1_PRODUCTION',
 'INTERLEUKIN_6_PRODUCTION']

In [9]:
gene_set_df=adata.obs[["sample","stage","celltype_L3"]+ gene_set_names]
#gene_set_df=adata.obs[["sample","stage"]+ gene_set_names]

In [10]:
gene_set_df

,sample,stage,celltype_L3,GOBP_CELL_CHEMOTAXIS,GOBP_INFLAMMATORY_RESPONSE,HALLMARK_INFLAMMATORY_RESPONSE,HALLMARK_INTERFERON_ALPHA_RESPONSE,HALLMARK_INTERFERON_GAMMA_RESPONSE,HALLMARK_TNFA_SIGNALING_VIA_NFKB,INTERLEUKIN_1_PRODUCTION,INTERLEUKIN_6_PRODUCTION
cellbarcode,,,,,,,,,,,
HD-H134-1-CELL5_N1,HD-H134,HDs,cMono-CD14,0.108496,0.142123,0.212276,0.286814,0.360184,0.167971,0.500167,0.468722
HD-H134-1-CELL10_N1,HD-H134,HDs,ncMono-FCGR3A,0.050202,0.066565,0.054229,0.728966,0.491731,0.164567,0.019661,0.197880
HD-H134-1-CELL30_N1,HD-H134,HDs,cMono-CD14,0.080163,0.119341,0.190124,0.203647,0.288333,0.249173,0.243611,0.108895
HD-H134-1-CELL33_N1,HD-H134,HDs,ncMono-FCGR3A,0.070112,0.071674,0.214493,0.359466,0.289371,0.110659,0.156043,0.170005
HD-H134-1-CELL45_N1,HD-H134,HDs,cMono-CD14,0.158889,0.075474,0.161508,0.210658,0.364277,0.254844,0.089910,0.123374
...,...,...,...,...,...,...,...,...,...,...,...
PD-H267-2-CELL11260_N1,PD-H267,IRs,pDC-LILRA4,0.036807,0.112780,0.115829,0.212415,0.351980,0.032644,0.158355,0.182042
PD-H267-2-CELL11275_N1,PD-H267,IRs,cMono-CD14,0.192186,0.134804,0.245202,0.037690,0.371647,0.341731,0.176554,0.212711
PD-H267-2-CELL11288_N1,PD-H267,IRs,cMono-IFI44L,0.176756,0.167404,0.282711,0.544475,0.552453,0.331676,0.346824,0.421626


In [11]:
gene_set_df.to_csv('/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/PathwayScore/数据导出/Myeloid/Myeloid-0302.csv', index=False)